# Import and Install Dependencies


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


In [4]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) #draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) #draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(232, 190, 172), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(232, 190, 172), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=2)
                             ) 

In [6]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #read feed 
        ret,frame=cap.read()

        #Make Detections
        image,results=mediapipe_detection(frame,holistic)
        
        #Draw landmarks
        draw_styled_landmarks(image,results)

        #show the screen
        cv2.imshow('OpenCV Feed', image)

        #break and escape from window
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
len(results.left_hand_landmarks.landmark)

21

In [8]:
results

mediapipe.python.solution_base.SolutionOutputs

In [9]:
draw_landmarks(frame, results)

# Extract Keypoint Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Setup Folders for Collection

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Words_Data') 

# Actions that we try to detect
actions = np.array(['Good', 'Bad', 'Yes','No','Sorry','Please','Thankyou','Welcome','I Love You','Hello','Bye','Namaste'])

# Thirty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect Keypoint Values for Training and Testing

In [9]:
def collect_keypoints(actions):
    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        # NEW LOOP
        # Loop through actions
        for action in actions:
            # Loop through sequences aka videos
            for sequence in range(no_sequences):
                # Loop through video length aka sequence length
                for frame_num in range(sequence_length):
                    if sequence>=0:
                        # Read feed
                        ret, frame = cap.read()

                        # Make detections
                        image, results = mediapipe_detection(frame, holistic)
#                         print(results)

                        # Draw landmarks
                        draw_styled_landmarks(image, results)

                        # NEW Apply wait logic
                        if frame_num == 0: 
                            cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            # Show to screen
                            cv2.imshow('OpenCV Feed', image)
                            cv2.waitKey(2000)
                        else: 
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            # Show to screen
                            cv2.imshow('OpenCV Feed', image)

                        # NEW Export keypoints
                        keypoints = extract_keypoints(results)
                        npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                        np.save(npy_path, keypoints)

                        # Break gracefully
                        if cv2.waitKey(10) & 0xFF == ord('q'):
                            break

        cap.release()
        cv2.destroyAllWindows()

In [11]:
# set1= np.array(['Good'])
# collect_keypoints(set1)

In [15]:
# set2= np.array(['Bad'])
# collect_keypoints(set2)

In [17]:
# set3= np.array(['Yes'])
# collect_keypoints(set3)

In [18]:
# set4= np.array(['No'])
# collect_keypoints(set4)

In [28]:
# set5= np.array(['Sorry'])
# collect_keypoints(set5)

In [29]:
# set6= np.array(['Please'])
# collect_keypoints(set6)

In [19]:
# set7= np.array(['Thankyou'])
# collect_keypoints(set7)

In [20]:
# set8= np.array(['Welcome'])
# collect_keypoints(set8)

In [30]:
# set9= np.array(['I Love You'])
# collect_keypoints(set9)

In [22]:
# set10= np.array(['Hello'])
# collect_keypoints(set10)

In [23]:
# set11= np.array(['Bye'])
# collect_keypoints(set11)

In [24]:
# set12= np.array(['Namaste'])
# collect_keypoints(set12)

# Preprocess Data and Create Labels and Features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [13]:
label_map = {label:num for num, label in enumerate(actions)}

In [14]:
label_map

{'Good': 0,
 'Bad': 1,
 'Yes': 2,
 'No': 3,
 'Sorry': 4,
 'Please': 5,
 'Thankyou': 6,
 'Welcome': 7,
 'I Love You': 8,
 'Hello': 9,
 'Bye': 10,
 'Namaste': 11}

In [15]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
np.array(sequences).shape

(600, 30, 1662)

In [17]:
np.array(labels).shape

(600,)

In [18]:
X = np.array(sequences)

In [19]:
X.shape

(600, 30, 1662)

In [20]:
y = to_categorical(labels).astype(int)

In [21]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [55]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,random_state=42)

In [23]:
y_test.shape


(300, 12)

# Build and Train LSTM Neural Network

In [11]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [25]:
log_dir = os.path.join('Logs2')
tb_callback = TensorBoard(log_dir=log_dir)

In [26]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [62]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
10/10 [==============================] - 3s 56ms/step - loss: 2.8253 - categorical_accuracy: 0.0767
Epoch 2/2000
10/10 [==============================] - 1s 57ms/step - loss: 2.5354 - categorical_accuracy: 0.1133
Epoch 3/2000
10/10 [==============================] - 1s 58ms/step - loss: 2.4889 - categorical_accuracy: 0.1100
Epoch 4/2000
10/10 [==============================] - 1s 53ms/step - loss: 2.4464 - categorical_accuracy: 0.1167
Epoch 5/2000
10/10 [==============================] - 1s 59ms/step - loss: 2.4605 - categorical_accuracy: 0.1467
Epoch 6/2000
10/10 [==============================] - 1s 53ms/step - loss: 2.4088 - categorical_accuracy: 0.1233
Epoch 7/2000
10/10 [==============================] - 1s 61ms/step - loss: 2.4361 - categorical_accuracy: 0.1267
Epoch 8/2000
10/10 [==============================] - 1s 60ms/step - loss: 2.2747 - categorical_accuracy: 0.1367
Epoch 9/2000
10/10 [==============================] - 1s 54ms/step - loss: 2.2019 - categorical_

10/10 [==============================] - 1s 62ms/step - loss: 0.9386 - categorical_accuracy: 0.6367
Epoch 144/2000
10/10 [==============================] - 1s 56ms/step - loss: 0.7934 - categorical_accuracy: 0.6800
Epoch 145/2000
10/10 [==============================] - 1s 60ms/step - loss: 0.7713 - categorical_accuracy: 0.7167
Epoch 146/2000
10/10 [==============================] - 1s 54ms/step - loss: 0.7236 - categorical_accuracy: 0.7400
Epoch 147/2000
10/10 [==============================] - 1s 62ms/step - loss: 0.7673 - categorical_accuracy: 0.7200
Epoch 148/2000
10/10 [==============================] - 1s 58ms/step - loss: 0.7010 - categorical_accuracy: 0.7333
Epoch 149/2000
10/10 [==============================] - 1s 56ms/step - loss: 0.7337 - categorical_accuracy: 0.7233
Epoch 150/2000
10/10 [==============================] - 1s 60ms/step - loss: 0.6548 - categorical_accuracy: 0.7433
Epoch 151/2000
10/10 [==============================] - 1s 54ms/step - loss: 0.8046 - categoric

KeyboardInterrupt: 

In [63]:
model.save('words1.h5')

In [28]:
models=tensorflow.keras.models.load_model('words1.h5')

In [66]:
models.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
10/10 [==============================] - 2s 65ms/step - loss: 0.3894 - categorical_accuracy: 0.8300
Epoch 2/2000
10/10 [==============================] - 1s 62ms/step - loss: 0.3347 - categorical_accuracy: 0.8800
Epoch 3/2000
10/10 [==============================] - 1s 56ms/step - loss: 0.4139 - categorical_accuracy: 0.8500
Epoch 4/2000
10/10 [==============================] - 1s 64ms/step - loss: 0.4706 - categorical_accuracy: 0.7967
Epoch 5/2000
10/10 [==============================] - 1s 57ms/step - loss: 0.3518 - categorical_accuracy: 0.8700
Epoch 6/2000
10/10 [==============================] - 1s 63ms/step - loss: 0.3292 - categorical_accuracy: 0.8667
Epoch 7/2000
10/10 [==============================] - 1s 66ms/step - loss: 0.4012 - categorical_accuracy: 0.8700
Epoch 8/2000
10/10 [==============================] - 1s 56ms/step - loss: 0.3312 - categorical_accuracy: 0.8867
Epoch 9/2000
10/10 [==============================] - 1s 62ms/step - loss: 0.3214 - categorical_

KeyboardInterrupt: 

In [67]:
models.save('wordsfinal.h5')

In [12]:
new=tensorflow.keras.models.load_model('wordsfinal.h5')

In [69]:
new.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_13 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_14 (LSTM)              (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 12)                396       
                                                                 
Total params: 596,972
Trainable params: 596,972
Non-tr

In [70]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [71]:
yhat = new.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [72]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[266,   4],
        [ 10,  20]],

       [[261,  10],
        [  5,  24]],

       [[267,  11],
        [  7,  15]],

       [[259,  16],
        [  9,  16]],

       [[276,   0],
        [  0,  24]],

       [[271,   3],
        [  7,  19]],

       [[274,   2],
        [  6,  18]],

       [[271,   7],
        [  3,  19]],

       [[266,   6],
        [  7,  21]],

       [[278,   2],
        [  2,  18]],

       [[277,   2],
        [  8,  13]],

       [[270,   1],
        [  0,  29]]], dtype=int64)

In [73]:
accuracy_score(ytrue, yhat)

0.7866666666666666

In [14]:
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = new.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
         #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 1: 
                sentence = sentence[-1:]
                
            cv2.putText(image, ' '.join(sentence), (25,80), cv2.FONT_HERSHEY_SIMPLEX, 2, (112, 41, 99), 6, cv2.LINE_AA)
            cv2.putText(image,'Press q to Exit', (460,470), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (112, 41, 99), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Love You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Love You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Yes
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Please
<class 'mediapipe.python.solution

Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Welcome
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Welcome
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Welcome
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Welcome
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Welcome
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Welcome
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Welcome
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Namaste
<class 'mediapipe.pyth

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Thankyou
<class 'me